In [1]:
import pandas as pd
import numpy as np
import random 

mean1=np.genfromtxt('DS1_m_0.txt',delimiter=',', usecols=range(0,20))
mean2=np.genfromtxt('DS1_m_1.txt',delimiter=',', usecols=range(0,20))
cov=np.genfromtxt('DS1_Cov.txt',delimiter=',', usecols=range(0,20))
#print(mean1.shape)
#print(mean2.shape)
#print(cov.shape)

In [2]:
# generating data 
negative = np.random.multivariate_normal(mean1, cov, 2000)
positive = np.random.multivariate_normal(mean2, cov, 2000)

#print(negative.shape)
#print(positive.shape)

np.random.shuffle(negative)
np.random.shuffle(positive)

test_neg = negative[:400]
test_pos = positive[:400]
valid_neg = negative[400:800]
valid_pos = positive[400:800]
train_neg = negative[800:]
train_pos = positive[800:]

# saving data 
train = np.vstack((train_neg, train_pos))
valid = np.vstack((valid_neg, valid_pos))
test = np.vstack((test_neg, test_pos))
train=train.transpose()
valid=valid.transpose()
test=test.transpose()
#print(test.shape)

np.savetxt('DS1_test.txt', test, delimiter=',')
np.savetxt('DS1_valid.txt', valid, delimiter=',')
np.savetxt('DS1_train.txt', train, delimiter=',')

In [3]:
train = np.asmatrix(np.genfromtxt('DS1_train.txt', delimiter=','))

# split data equally to positive and negative 
train_neg = train[:,:1200]
train_pos = train[:,1200:]

# Bernoulli distribution 
p0 = train_neg.shape[1]/(train_neg.shape[1]+train_pos.shape[1])
p1 = train_pos.shape[1]/(train_neg.shape[1]+train_pos.shape[1])
print("P(C1):", p0)
print("P(C2):",p1)

size1 = train_neg.shape[1]
size2 = train_pos.shape[1]
total_size = size1+size2
# getting the mean    
mean_neg = np.sum(train_neg,axis=1)/size1
mean_neg = mean_neg.reshape(20,1)
mean_pos = np.sum(train_pos,axis=1)/size2
mean_pos = mean_pos.reshape(20,1)
print("Mean for negative class:", mean_neg)
print("Mean for positive class:",mean_pos)

P(C1): 0.5
P(C2): 0.5
Mean for negative class: [[1.40678156]
 [1.35621231]
 [1.34121479]
 [1.31092959]
 [1.33147247]
 [1.28766246]
 [1.32792318]
 [1.34234643]
 [1.32209534]
 [1.30148562]
 [1.37894101]
 [1.34905823]
 [1.34575802]
 [1.40210804]
 [1.31273688]
 [1.25216984]
 [1.34289486]
 [1.36553229]
 [1.39307692]
 [1.26229694]]
Mean for positive class: [[1.94902307]
 [1.94435908]
 [1.99891726]
 [1.98060256]
 [2.00887559]
 [1.95362731]
 [2.00858526]
 [1.9489678 ]
 [1.98912591]
 [1.9810488 ]
 [1.95527848]
 [1.99760139]
 [1.98676203]
 [1.92993441]
 [1.93206283]
 [1.89485157]
 [1.97858986]
 [1.97567225]
 [1.94194911]
 [1.88862365]]


In [4]:
s1 = np.asmatrix(train_neg-mean_neg).dot(np.asmatrix(train_neg-mean_neg).transpose())
s2 = np.asmatrix(train_pos-mean_pos).dot(np.asmatrix(train_pos-mean_pos).transpose())
s_total = s1/(total_size) + s2/(total_size)
print("Covariance=", s_total) 

Covariance= [[8.21084177 5.81045983 6.51574353 5.40659162 6.07448703 6.48302518
  4.91372042 5.69846994 5.13556349 5.35263059 3.97659021 5.43830762
  7.33211395 6.1882475  6.40615365 6.30199413 6.08534656 5.91464206
  5.8280976  6.10541077]
 [5.81045983 7.17781434 5.74968339 4.56214706 5.76352709 5.88168893
  4.53816627 4.11470624 4.30959546 5.13937907 3.50247095 4.77494712
  6.12664193 5.35510537 5.76422015 5.48743818 5.92171698 5.36872245
  5.59098102 5.49773584]
 [6.51574353 5.74968339 7.62760714 5.08124055 6.05357908 6.93364037
  4.85339131 4.9832009  5.04621509 5.22588505 3.38983594 4.83609914
  6.57495273 5.36407794 6.44015293 6.26428558 6.49615562 5.21494202
  4.96356816 5.29507203]
 [5.40659162 4.56214706 5.08124055 5.94348157 5.48486855 4.68418495
  3.91453446 4.53574611 3.50945405 4.30633176 2.84881275 4.29772834
  6.01494385 4.96338172 5.06421353 5.32573905 4.84467887 4.72201754
  4.12071403 5.89741384]
 [6.07448703 5.76352709 6.05357908 5.48486855 7.22049589 5.62528506
  5.

In [5]:
w= np.dot(np.linalg.inv(s_total),(mean_neg-mean_pos))
w0=(-1/2)*(mean_neg.transpose().dot(np.linalg.inv(s_total).dot(mean_neg)))+(1/2)*((mean_pos.transpose().dot(np.linalg.inv(s_total).dot(mean_pos)))) + np.log(p0/p1)                                                                                           

print("W1:",w)
print("W0:", w0)

W1: [[ 13.60822853]
 [ -8.21588776]
 [ -6.03829415]
 [ -3.48357102]
 [ -9.40172307]
 [ -3.77850091]
 [ 16.92858653]
 [-22.58114781]
 [-28.17806632]
 [  8.36317307]
 [-12.14562283]
 [-12.02679586]
 [ 15.10583801]
 [ 12.61281365]
 [ -5.15860202]
 [ 12.20128768]
 [ 28.68309467]
 [ -6.17024194]
 [ -1.42129265]
 [ -4.5914406 ]]
W0: [[25.85056091]]


In [6]:
### decision boundary 
def logsig(a): 
    return 1/(1+np.exp(-a))

In [7]:
def class1(x,w,w0):
    y = logsig(w.transpose().dot(x)+w0)
    classify1 = np.zeros(x.shape[1])
    classify1[np.where(y<0.5)[1]]=1
    return classify1

In [8]:
test = np.asmatrix(np.genfromtxt('DS1_test.txt', delimiter=','))
class2=class1(test,w,w0)

In [9]:
neg1=class2[:400]
pos1=class2[400:]

In [10]:
trueneg = len(np.where(neg1 == 0)[0]) 
falsepos = len(np.where(neg1 == 1)[0]) 
truepos = len(np.where(pos1 == 1)[0]) 
falseneg = len(np.where(pos1 == 0)[0]) 
print(trueneg, falsepos, truepos, falseneg)

376 24 381 19


In [11]:
def decisionmatrix(trueneg, falsepos, truepos, falseneg):
    acc = (trueneg+truepos)/(trueneg+falseneg+truepos+falsepos)
    prec = truepos/(truepos+falsepos)
    rec = truepos/(truepos+falseneg)
    fmeasure = 2*prec*rec/(prec+rec)
    return acc, prec, rec, fmeasure

acc, prec, rec, fmeasure = decisionmatrix(trueneg,falsepos,truepos,falseneg)

print("For Dataset1:")
print("Accuracy:",acc)
print("Precision:",prec)
print("Recall:",rec)
print("F1 Measure:",fmeasure)

For Dataset1:
Accuracy: 0.94625
Precision: 0.9407407407407408
Recall: 0.9525
F1 Measure: 0.9465838509316771


In [12]:
train = np.asmatrix(np.genfromtxt('DS1_train.txt', delimiter=','))
valid = np.asmatrix(np.genfromtxt('DS1_valid.txt', delimiter=','))
test = np.asmatrix(np.genfromtxt('DS1_test.txt', delimiter=','))
#print(train.shape)
#print(test.shape)

In [13]:
# calculates distance between each row of training and test data 
def euclideanDist(train, test):
    distance=np.sum(np.square(np.subtract(train,test)),axis=0)
    return np.sqrt(distance)  

In [14]:
def classify(distancematrix, k): 
    neighbours = distancematrix[:, :k]
    neighbours[np.where(neighbours<1200)]=0
    neighbours[np.where(neighbours>=1200)]=1
    average = np.sum(neighbours)/(neighbours.shape[1])
    x=0
    if (average>0.5): 
        x=1
    return x

In [15]:
# kNN model 
def knn(train, test, k): 
    kmatrix=np.zeros((k,test.shape[1]))
    for i in range(k):
        for x in range(test.shape[1]): 
            distance = euclideanDist(train,test[:,x])
            sorteddist = np.argsort(distance)
            kmatrix[i,x]=classify(sorteddist, i+1)
    return kmatrix

In [16]:
# good starting point for k is via using sqrt(n)
k=100
# store k nearest neighbours in kmatrix     
kmatrix=knn(train, valid, k)
#print(kmatrix)
np.random.shuffle(kmatrix)
kmatrix_pos = kmatrix[:, 400:] 
kmatrix_neg = kmatrix[:, :400]
#print(kmatrix_pos.shape)
#print(kmatrix_neg.shape)

In [17]:
acc_knn=np.zeros(k)
prec_knn=np.zeros(k)
rec_knn=np.zeros(k)
fmeasure_knn=np.zeros(k)

In [18]:
def accuracy(k, kmatrix_neg, kmatrix_pos):
    for i in range(k):
        trueneg_knn = len(np.where(kmatrix_neg[i,:] == 0)[0]) 
        falsepos_knn = len(np.where(kmatrix_neg[i,:] == 1)[0]) 
        truepos_knn = len(np.where(kmatrix_pos[i,:] == 1)[0]) 
        falseneg_knn = len(np.where(kmatrix_pos[i,:] == 0)[0]) 
        acc_knn[i] = (trueneg_knn+truepos_knn)/(trueneg_knn+falseneg_knn+truepos_knn+falsepos_knn)
        prec_knn[i] = truepos_knn/(truepos_knn+falsepos_knn)
        rec_knn[i] = truepos_knn/(truepos_knn+falseneg_knn)
        fmeasure_knn[i] = (2*prec_knn[i]*rec_knn[i])/(prec_knn[i]+rec_knn[i])
    return acc_knn, prec_knn, rec_knn, fmeasure_knn 

In [19]:
acc_knn, prec_knn, rec_knn, fmeasure_knn = accuracy(k, kmatrix_neg, kmatrix_pos)
result = np.vstack((acc_knn,prec_knn,rec_knn,fmeasure_knn)).transpose()
print("Result:",result)

Result: [[0.53125    0.53753754 0.4475     0.48840382]
 [0.55875    0.56582633 0.505      0.5336856 ]
 [0.5525     0.55072464 0.57       0.56019656]
 [0.55125    0.55086849 0.555      0.55292653]
 [0.5625     0.56038647 0.58       0.57002457]
 [0.55125    0.55380577 0.5275     0.54033291]
 [0.5425     0.54381443 0.5275     0.53553299]
 [0.54625    0.54683544 0.54       0.54339623]
 [0.55875    0.55662651 0.5775     0.56687117]
 [0.54375    0.54475703 0.5325     0.53855879]
 [0.555      0.55527638 0.5525     0.55388471]
 [0.54875    0.54961832 0.54       0.54476671]
 [0.5425     0.54427083 0.5225     0.53316327]
 [0.5025     0.50490196 0.2575     0.3410596 ]
 [0.5225     0.52238806 0.525      0.52369077]
 [0.5675     0.56852792 0.56       0.56423174]
 [0.56125    0.56265985 0.55       0.5562579 ]
 [0.53875    0.54466859 0.4725     0.5060241 ]
 [0.5575     0.55808081 0.5525     0.55527638]
 [0.535      0.53589744 0.5225     0.52911392]
 [0.55625    0.55667506 0.5525     0.55457967]
 [0.5

In [20]:
best_k = np.argmax(fmeasure_knn)
print(best_k)
########## USING VALIDATION, PICK BEST K 
kmatrix_test=knn(train, test, best_k)
kmatrix_test_neg = kmatrix_test[:, :400]
kmatrix_test_pos = kmatrix_test[:, 400:]   

84


In [21]:
trueneg_test = len(np.where(kmatrix_test_neg == 0)[0]) 
falsepos_test = len(np.where(kmatrix_test_neg == 1)[0]) 
truepos_test = len(np.where(kmatrix_test_pos == 1)[0]) 
falseneg_test = len(np.where(kmatrix_test_pos == 0)[0]) 
#print(trueneg_test, falsepos_test, truepos_test, falseneg_test)

In [22]:
acc_test, prec_test, rec_test, fmeasure_test = decisionmatrix(trueneg_test, falsepos_test, truepos_test, falseneg_test)

print("For Dataset1_KNN:")
print("Accuracy:",acc_test)
print("Precision:",prec_test)
print("Recall:",rec_test)
print("F1 Measure:",fmeasure_test)

For Dataset1_KNN:
Accuracy: 0.5597470238095238
Precision: 0.5623854066316542
Recall: 0.5386011904761905
F1 Measure: 0.5502363976345033


In [23]:
mean_n1=np.genfromtxt('DS2_c2_m1.txt',delimiter=',', usecols=range(0,20))
mean_p1=np.genfromtxt('DS2_c1_m1.txt',delimiter=',', usecols=range(0,20))
cov1=np.genfromtxt('DS2_Cov1.txt',delimiter=',', usecols=range(0,20))
mean_n2=np.genfromtxt('DS2_c2_m2.txt',delimiter=',', usecols=range(0,20))
mean_p2=np.genfromtxt('DS2_c1_m2.txt',delimiter=',', usecols=range(0,20))
cov2=np.genfromtxt('DS2_Cov2.txt',delimiter=',', usecols=range(0,20))
mean_n3=np.genfromtxt('DS2_c2_m3.txt',delimiter=',', usecols=range(0,20))
mean_p3=np.genfromtxt('DS2_c1_m3.txt',delimiter=',', usecols=range(0,20))
cov3=np.genfromtxt('DS2_Cov3.txt',delimiter=',', usecols=range(0,20))

In [24]:
negative1 = np.random.multivariate_normal(mean_n1, cov1, 2000)
positive1 = np.random.multivariate_normal(mean_p1, cov1, 2000)
negative2 = np.random.multivariate_normal(mean_n2, cov2, 2000)
positive2 = np.random.multivariate_normal(mean_p2, cov2, 2000)
negative3 = np.random.multivariate_normal(mean_n3, cov3, 2000)
positive3 = np.random.multivariate_normal(mean_p3, cov3, 2000)

np.random.shuffle(negative1)
np.random.shuffle(negative2)
np.random.shuffle(negative3)
np.random.shuffle(positive1)
np.random.shuffle(positive2)
np.random.shuffle(positive3)

negative123=np.vstack((negative1[:200],negative2[:840],negative3[:960]))
positive123=np.vstack((positive1[:200],positive2[:840],positive3[:960]))


np.random.shuffle(negative123)
np.random.shuffle(positive123)
                      
test_neg1 = negative123[:400]
test_pos1 = positive123[:400]
valid_neg1 = negative123[400:800]
valid_pos1 = positive123[400:800]
train_neg1 = negative123[800:]
train_pos1 = positive123[800:]
                      
# saving data 
train = np.vstack((train_neg1, train_pos1))
valid = np.vstack((valid_neg1, valid_pos1))
test = np.vstack((test_neg1, test_pos1))
DS2 = np.vstack((train,valid,test)).transpose()
train=train.transpose()
valid=valid.transpose()
test=test.transpose()
#print(test.shape)
#print(train.shape)

np.savetxt('DS2_test.txt', test, delimiter=',')
np.savetxt('DS2_valid.txt', valid, delimiter=',')
np.savetxt('DS2_train.txt', train, delimiter=',')
np.savetxt('DS2.txt',DS2, delimiter=',')

                    

In [25]:
train1 = np.asmatrix(np.genfromtxt('DS2_train.txt', delimiter=','))
#print(train1.shape)
# split data equally to positive and negative 
train_neg1 = train1[:,:1200]
train_pos1 = train1[:,1200:]

# Bernoulli distribution 
p01 = train_neg1.shape[1]/(train_neg1.shape[1]+train_pos1.shape[1])
p11 = train_pos1.shape[1]/(train_neg1.shape[1]+train_pos1.shape[1])
print("P(C1):", p01)
print("P(C2):",p11)

size11 = train_neg1.shape[1]
size21 = train_pos1.shape[1]
total_size1 = size11+size21

# getting the mean    
mean_neg1 = (np.sum(train_neg1,axis=1)/size11).reshape(20,1)
mean_pos1 = (np.sum(train_pos1,axis=1)/size21).reshape(20,1)
print("Mean for negative class_DS2:", mean_neg1)
print("Mean for positive class_DS2:",mean_pos1)

P(C1): 0.5
P(C2): 0.5
Mean for negative class_DS2: [[1.15945546]
 [1.16438647]
 [1.11084744]
 [1.14350626]
 [1.16799865]
 [1.1266781 ]
 [1.14575689]
 [1.19845562]
 [1.14260439]
 [1.1371304 ]
 [1.12019077]
 [1.12490572]
 [1.06493517]
 [1.07997825]
 [1.11884202]
 [1.15282872]
 [1.09294206]
 [1.20980663]
 [1.09750151]
 [1.14893972]]
Mean for positive class_DS2: [[0.88977517]
 [0.83590452]
 [0.86251393]
 [0.92966219]
 [0.8726556 ]
 [0.8878994 ]
 [0.8303052 ]
 [0.89776702]
 [0.88975272]
 [0.94849961]
 [0.88028921]
 [0.92276132]
 [0.94945254]
 [0.87057591]
 [0.92644006]
 [0.85608308]
 [0.97932313]
 [0.88598625]
 [0.82167947]
 [0.88339815]]


In [26]:
s11 = np.asmatrix(train_neg1-mean_neg1).dot(np.asmatrix(train_neg1-mean_neg1).transpose())
s21 = np.asmatrix(train_pos1-mean_pos1).dot(np.asmatrix(train_pos1-mean_pos1).transpose()) 
s_total1 = s11/total_size1 + s21/total_size1
print("Covariance_DS2 is:",s_total1)

Covariance_DS2 is: [[7.95863594 5.56492099 4.71021435 5.10805404 4.31551833 5.89575196
  6.10463365 5.68313304 4.90100779 5.53506774 5.7642167  5.19084741
  5.26342646 6.2670544  5.48075891 5.74441744 5.49152307 5.64111181
  5.77217203 5.9235058 ]
 [5.56492099 7.35799244 4.99146501 5.27324197 5.0830105  6.13559797
  6.64356399 5.40875599 4.80174009 5.2280416  4.96444339 5.15307217
  5.02255921 6.14227008 5.72520316 6.0348085  5.35048668 5.22913926
  6.25093094 5.74375264]
 [4.71021435 4.99146501 6.77336692 5.23440936 4.79331785 4.99488148
  6.05982496 4.60998656 4.71859229 4.88645765 4.87792074 5.19243909
  5.17628104 6.06995194 5.47474765 4.96057582 5.43365129 4.52972064
  5.93685439 5.82987541]
 [5.10805404 5.27324197 5.23440936 6.61912838 4.25761936 5.62727023
  6.27612734 5.41122315 4.89807672 5.51148764 4.99154591 4.95337096
  4.99721292 5.53373292 5.36722166 5.38536022 5.19369816 5.27558657
  5.87702744 5.65559411]
 [4.31551833 5.0830105  4.79331785 4.25761936 5.63348136 4.752784

In [27]:
w_ds2=np.dot(np.linalg.inv(s_total),np.asmatrix(mean_neg1-mean_pos1))
w0_ds2=(-1/2)*(mean_neg1.transpose().dot(np.linalg.inv(s_total1)).dot(mean_neg1))+(1/2)*((mean_pos1.transpose().dot(np.linalg.inv(s_total1)).dot(mean_pos1)))+np.log(p01/p11)                                                                                           
print("W1_DS2:",w_ds2)
print("w0_DS2:", w0_ds2)

W1_DS2: [[-12.90307113]
 [  8.41931307]
 [  7.51954159]
 [  4.10046572]
 [ 10.07078801]
 [  2.94607961]
 [-18.72919453]
 [ 21.86830896]
 [ 28.92643441]
 [ -7.71393198]
 [ 11.1579141 ]
 [ 12.73829251]
 [-16.00739746]
 [-13.27694665]
 [  4.18759028]
 [-11.44060363]
 [-30.09109372]
 [  5.72921484]
 [  3.26796127]
 [  4.47871591]]
w0_DS2: [[-0.05358927]]


In [28]:

test_ds2 = np.asmatrix(np.genfromtxt('DS2_test.txt', delimiter=','))
class2_ds2=class1(test_ds2,w_ds2,w0_ds2)
negative_ds2=class2_ds2[:400]
positive_ds2=class2_ds2[400:]

trueneg_ds2 = len(np.where(negative_ds2 == 0)[0]) 
falsepos_ds2 = len(np.where(negative_ds2 == 1)[0]) 
truepos_ds2 = len(np.where(positive_ds2 == 1)[0]) 
falseneg_ds2 = len(np.where(positive_ds2 == 0)[0]) 
print(trueneg_ds2, falsepos_ds2, truepos_ds2, falseneg_ds2)


249 151 174 226


In [29]:
acc_ds2,prec_ds2,rec_ds2,fmeasure_ds2 = decisionmatrix(trueneg_ds2, falsepos_ds2, truepos_ds2, falseneg_ds2)

print("For Dataset 2_GDA:")
print("Accuracy:",acc_ds2)
print("Precision:",prec_ds2)
print("Recall:",rec_ds2)
print("F1 Measure:",fmeasure_ds2)

For Dataset 2_GDA:
Accuracy: 0.52875
Precision: 0.5353846153846153
Recall: 0.435
F1 Measure: 0.48


In [30]:
###### KNN for DS2 
train_ds2 = np.asmatrix(np.genfromtxt('DS2_train.txt', delimiter=','))
valid_ds2 = np.asmatrix(np.genfromtxt('DS2_valid.txt', delimiter=','))
test_ds2 = np.asmatrix(np.genfromtxt('DS2_test.txt', delimiter=','))


In [31]:
k=100
# store k nearest neighbours in kmatrix     
kmatrix_ds2=knn(train_ds2, valid_ds2, k)
#print(kmatrix)
np.random.shuffle(kmatrix_ds2)
kmatrix_neg_ds2 = kmatrix_ds2[:, :400]
kmatrix_pos_ds2 = kmatrix_ds2[:, 400:] 

In [32]:
acc_ds2_knn=np.zeros(k)
prec_ds2_knn=np.zeros(k)
rec_ds2_knn=np.zeros(k)
fmeasure_ds2_knn=np.zeros(k)

In [33]:
for i in range(k):
    trueneg_ds2_knn = len(np.where(kmatrix_neg_ds2[i,:] == 0)[0]) 
    falsepos_ds2_knn = len(np.where(kmatrix_neg_ds2[i,:] == 1)[0]) 
    truepos_ds2_knn = len(np.where(kmatrix_pos_ds2[i,:] == 1)[0]) 
    falseneg_ds2_knn = len(np.where(kmatrix_pos_ds2[i,:] == 0)[0]) 
    acc_ds2_knn[i] = (trueneg_ds2_knn+truepos_ds2_knn)/(trueneg_ds2_knn+falseneg_ds2_knn+truepos_ds2_knn+falsepos_ds2_knn)
    prec_ds2_knn[i] = truepos_ds2_knn/(truepos_ds2_knn+falsepos_ds2_knn)
    rec_ds2_knn[i] = truepos_ds2_knn/(truepos_ds2_knn+falseneg_ds2_knn)
    fmeasure_ds2_knn[i] = (2*prec_ds2_knn[i]*rec_ds2_knn[i])/(prec_ds2_knn[i]+rec_ds2_knn[i])

In [34]:
result = np.vstack((acc_ds2_knn,prec_ds2_knn,rec_ds2_knn,fmeasure_ds2_knn)).transpose()
print("Result_DS2:",result)

Result_DS2: [[0.52125    0.52380952 0.4675     0.49405548]
 [0.545      0.53947368 0.615      0.57476636]
 [0.545      0.54166667 0.585      0.5625    ]
 [0.525      0.52538071 0.5175     0.52141058]
 [0.5475     0.54398148 0.5875     0.56490385]
 [0.54125    0.53828306 0.58       0.55836342]
 [0.5375     0.53246753 0.615      0.57076566]
 [0.5425     0.53744493 0.61       0.57142857]
 [0.54125    0.53758542 0.59       0.56257449]
 [0.545      0.54147465 0.5875     0.56354916]
 [0.53125    0.52913753 0.5675     0.54764777]
 [0.5225     0.52195122 0.535      0.52839506]
 [0.52625    0.52861035 0.485      0.50586701]
 [0.535      0.5292887  0.6325     0.57630979]
 [0.52875    0.5335277  0.4575     0.49259758]
 [0.55625    0.54880694 0.6325     0.58768873]
 [0.54625    0.54373522 0.575      0.55893074]
 [0.51625    0.51460674 0.5725     0.54201183]
 [0.51625    0.51800554 0.4675     0.49145861]
 [0.54       0.5372093  0.5775     0.55662651]
 [0.52875    0.52527473 0.5975     0.55906433]
 

In [35]:
best_k2 = np.argmax(fmeasure_ds2_knn)
print(best_k2)
############### PERFORMANCE BASED ON K IN VALIDATION 
kmatrix_test2=knn(train_ds2, test_ds2, best_k2)

np.random.shuffle(kmatrix_test2)
kmatrix_test2_neg = kmatrix_test2[:, :400]
kmatrix_test2_pos = kmatrix_test2[:, 400:]

trueneg_test2 = len(np.where(kmatrix_test2_neg == 0)[0]) 
falsepos_test2 = len(np.where(kmatrix_test2_neg == 1)[0]) 
truepos_test2 = len(np.where(kmatrix_test2_pos == 1)[0]) 
falseneg_test2 = len(np.where(kmatrix_test2_pos == 0)[0]) 
print(trueneg_test2, falsepos_test2, truepos_test2, falseneg_test2)

acc_test2,prec_test2,rec_test2,fmeasure_test2 = decisionmatrix(trueneg_test2,falsepos_test2,truepos_test2, falseneg_test2)

print("For Dataset2_KNN:")
print("Accuracy:",acc_test2)
print("Precision:",prec_test2)
print("Recall:",rec_test2)
print("F1 Measure:",fmeasure_test2)

15
3324 2676 2668 3332
For Dataset2_KNN:
Accuracy: 0.49933333333333335
Precision: 0.49925149700598803
Recall: 0.44466666666666665
F1 Measure: 0.4703808180535966
